In [57]:
# Parameters
length = 8   # Number of holes on each side
mirror_positions = [(3,2),(3,7),(6,4),(8,7,10)]    # Mirror positions
rays = ['C2+','C3+','C5+','C6-','R4+','R5+','C7+','R5-'] # ['C7+','C5+','R5-']

In [58]:
# Cell Class
class Cell():
    def __init__(self) -> None:
        self.mirror = False
        self.life = float('inf')

In [59]:
def createGrid():
    # Create Empty Grid
    rows = []
    for _ in range(length):
        columns = []
        for _ in range(length):
            columns.append(Cell())
        rows.append(columns)
    print(rows)

    # Insert Mirrors
    for mirror in mirror_positions:
        row_idx = mirror[0] - 1
        col_idx = mirror[1] - 1
        cell = rows[row_idx][col_idx]
        cell.mirror = True
        if len(mirror) == 3:
            cell.life = mirror[2] 
    return rows

In [60]:
rows = createGrid()

[[<__main__.Cell object at 0x000002A379887BD0>, <__main__.Cell object at 0x000002A3798E3210>, <__main__.Cell object at 0x000002A379887510>, <__main__.Cell object at 0x000002A3799D5A10>, <__main__.Cell object at 0x000002A3799D6D90>, <__main__.Cell object at 0x000002A3799D5050>, <__main__.Cell object at 0x000002A3799D5690>, <__main__.Cell object at 0x000002A3799D5810>], [<__main__.Cell object at 0x000002A3799D6090>, <__main__.Cell object at 0x000002A3799D5390>, <__main__.Cell object at 0x000002A3799D5790>, <__main__.Cell object at 0x000002A3799D41D0>, <__main__.Cell object at 0x000002A3799D5910>, <__main__.Cell object at 0x000002A3799D5D10>, <__main__.Cell object at 0x000002A3799D5E50>, <__main__.Cell object at 0x000002A3799D5A50>], [<__main__.Cell object at 0x000002A3799D5990>, <__main__.Cell object at 0x000002A3799D5B50>, <__main__.Cell object at 0x000002A3799D5290>, <__main__.Cell object at 0x000002A3799D5C10>, <__main__.Cell object at 0x000002A3799D5450>, <__main__.Cell object at 0x0

In [118]:
# Beam Class
class Beam():
    def __init__(self,start_coordinates,length) -> None:
        self.start_coordinates = start_coordinates
        self.end = False
        # Initialise the starting coordinates of the beam
        if start_coordinates[0]=='R' and start_coordinates[2]=='+':
            self.curr_row_idx = int(start_coordinates[1]) - 1 
            self.curr_col_idx = 0
            self.current_direction = 'right'
        elif start_coordinates[0]=='R' and start_coordinates[2]=='-':
            self.curr_row_idx = int(start_coordinates[1]) - 1
            self.curr_col_idx = length - 1
            self.current_direction = 'left'    
        elif start_coordinates[0]=='C' and start_coordinates[2]=='+':
            self.curr_row_idx = 0
            self.curr_col_idx = int(start_coordinates[1]) - 1
            self.current_direction = 'down'
        elif start_coordinates[0]=='C' and start_coordinates[2]=='-':
            self.curr_row_idx = length - 1
            self.curr_col_idx = int(start_coordinates[1]) - 1
            self.current_direction = 'up'

        print('Original Beam Position:',self.current_direction,self.curr_row_idx+1,self.curr_col_idx+1)

        # If mirror right beside end immediately
        if start_coordinates[0]=='C':
            if not (self.curr_col_idx == 0):
                if rows[self.curr_row_idx][self.curr_col_idx - 1].mirror:
                    self.end = True
                    self.print_output()
            if not (self.curr_col_idx == (length - 1)):
                if rows[self.curr_row_idx][self.curr_col_idx + 1].mirror:
                    self.end = True
                    self.print_output()
        if start_coordinates[0]=='R':
            if not (self.curr_row_idx == 0):
                if rows[self.curr_row_idx - 1][self.curr_col_idx].mirror:
                    self.end = True
                    self.print_output()
            if not (self.curr_row_idx == (length - 1)):
                if rows[self.curr_row_idx + 1][self.curr_col_idx].mirror:
                    self.end = True
                    self.print_output()
    
    def check_collision(self):
        if self.end:
            return None
        # Check if current cell is a mirror
        curr_grid_cell = rows[self.curr_row_idx][self.curr_col_idx]
        if curr_grid_cell.mirror == True:
            self.current_direction = None
            self.end = True
            print(self.start_coordinates,'->')
            curr_grid_cell.life -= 1
            if curr_grid_cell.life == 0:
                curr_grid_cell.mirror = False
                curr_grid_cell.life = float('inf')
            return None

    def check_mirror(self):
        if self.end:
            return None
        top_right_mirror = False
        bottom_right_mirror = False
        top_left_mirror = False
        bottom_left_mirror = False

        # Check whether adjacent diagonals are mirrors and redirect
        if not (self.curr_col_idx == length - 1) and not (self.curr_row_idx == 0):
            top_right_mirror = rows[self.curr_row_idx-1][self.curr_col_idx+1].mirror
        if not (self.curr_col_idx == length - 1) and not (self.curr_row_idx == length - 1):
            bottom_right_mirror = rows[self.curr_row_idx+1][self.curr_col_idx+1].mirror
        if not (self.curr_col_idx == 0) and not (self.curr_row_idx == 0):
            top_left_mirror = rows[self.curr_row_idx-1][self.curr_col_idx-1].mirror
        if not (self.curr_col_idx == 0) and not (self.curr_row_idx == length - 1):
            bottom_left_mirror = rows[self.curr_row_idx+1][self.curr_col_idx-1].mirror

        if self.current_direction  == 'right':
            if top_right_mirror and bottom_right_mirror:
                self.current_direction = 'left'
            elif  top_right_mirror:
                self.current_direction = 'down'
            elif bottom_right_mirror:
                self.current_direction = 'up'

        elif self.current_direction  == 'left':
            if top_left_mirror and bottom_left_mirror:
                self.current_direction = 'right'
            elif top_left_mirror:
                self.current_direction = 'down'
            elif  bottom_left_mirror:
                self.current_direction = 'up'

        elif self.current_direction  == 'up':
            if top_left_mirror and top_right_mirror:
                self.current_direction = 'down'
            elif top_left_mirror:
                self.current_direction = 'right'
            elif top_right_mirror:
                self.current_direction = 'left'

        elif self.current_direction  == 'down':
            if bottom_left_mirror and bottom_right_mirror:
                self.current_direction = 'up'
            elif bottom_left_mirror:
                self.current_direction = 'right'
            elif bottom_right_mirror:
                self.current_direction = 'left'
    
    def print_output(self):
        print(self.start_coordinates,'->','{'+str(self.curr_row_idx + 1)+','+str(self.curr_col_idx + 1)+'}')

    def check_end(self):
        if self.current_direction == 'up' and self.curr_row_idx == 0:
            self.end = True
            self.print_output()
        elif self.current_direction == 'right' and self.curr_col_idx == (length - 1):
            self.end = True
            self.print_output()
        elif self.current_direction == 'down' and self.curr_row_idx == (length - 1):
            self.end = True
            self.print_output()    
        elif self.current_direction == 'left' and self.curr_col_idx == 0:
            self.end = True
            self.print_output()    
    
    def move(self):
        if self.end:
            return None
        # Move beam one step in current direction
        if self.current_direction  == 'right':
            self.curr_col_idx += 1
        elif self.current_direction  == 'left':
            self.curr_col_idx -= 1
        elif self.current_direction  == 'up':
            self.curr_row_idx -= 1
        elif self.current_direction  == 'down':
            self.curr_row_idx += 1

In [91]:
rows[5][3].mirror

True

In [105]:
not (beam1.curr_col_idx == length - 1)

False

In [95]:
beam1 = Beam(rays[3],length)
beam1.check_end()
beam1.check_mirror()
beam1.move()
beam1.check_collision()
print(beam1.curr_row_idx+1,beam1.curr_col_idx+1)

7 6


In [119]:
for ray in rays:
    beam1 = Beam(ray,length)
    while not beam1.end:
        beam1.check_end()
        beam1.check_mirror()
        beam1.move()
        beam1.check_collision()
        print(beam1.curr_row_idx+1,beam1.curr_col_idx+1)

Original Beam Position: down 1 2
2 2
C2+ ->
3 2
Original Beam Position: down 1 3
2 3
2 4
2 5
2 6
1 6
C3+ -> {1,6}
1 6
Original Beam Position: down 1 5
2 5
3 5
4 5
5 5
5 6
5 7
5 8
C5+ -> {5,8}
5 8
Original Beam Position: up 8 6
C6- -> {8,6}
Original Beam Position: right 4 1
5 1
6 1
7 1
8 1
R4+ -> {8,1}
8 1
Original Beam Position: right 5 1
5 2
5 3
4 3
4 4
4 5
4 6
5 6
6 6
7 6
7 5
8 5
R5+ -> {8,5}
8 5
Original Beam Position: down 1 7
2 7
C7+ ->
3 7
Original Beam Position: left 5 8
5 7
5 6
5 5
4 5
3 5
2 5
1 5
R5- -> {1,5}
1 5


In [74]:
rays[0][0]

'C'

In [110]:
beam1.curr_col_idx

7

In [112]:
print(not (beam1.curr_col_idx == (length - 1)))

False


In [44]:
rows[beam1.curr_row_idx+1][beam1.curr_col_idx-1].mirror

True

In [ ]:
def main():
    """
    Manage application dependencies
    """
    config_logger()
    logger.info("Configured logger")

    consumer = XXX
    conumer.run()


if __main__ == '__main__':
    # parse and verify the input (abstract input validation with another function or file)
    try:
        main()
    except KeyboardInterrupt:
        logger.info('Interrupted')
        try:
            sys.exit()
        except SystemExit:
            os._exit(0)